In [1]:
%run stdPackages.ipynb
from lpEnergyModels.mBasic import * # Load everything from the mBasic module

*Note: "`self`" is a placeholder that refers to the class MBasic or a specific instance of the class.*

# How to perform simple shocks/simulations

The notebook shows how to do simple policy shocks/simulations in the model. Specifically, we show how to set up a loop that updates some exogenous inputs to the model (e.g. tax lavels on emissions), solve, and store the results. We show a simple "lazy" approach and outline how to do this more efficiently (with big models/many simulations).

We build on the `MBasic` model (see [notebook](MBasic.ipynb) for description of the model) and start by loading data, initializing model, and compiling the baseline structure:

In [2]:
testData = os.path.join(_d['data'],'EX_MBasic.xlsx')
data = pyDbs.ExcelSymbolLoader(testData)() # read data in a dict 
m = MBasic() # initialize model
[m.db.__setitem__(k, v) for k,v in data.items() if k!='__meta__']; # add data to model database
m.compile(); # create model structure

The `MBasic` model relies on a tax on CO2 emissions `taxEm` that enters the model through the marginal costs of generators (`mc`). To start with, we define a grid of taxes that we want to run the model through:

In [3]:
name = 'taxEm' # name of model variable
v0 = pd.Series(0, index = m.db(name).index) # Lowest value to look at
vT = pd.Series(250, index = m.db(name).index) # Highest value to solve for 
idxLoop = pd.Index(range(11), name = 'idxLoop') # an index we can use to loop through
grid = adjMultiIndex.addGrid(v0, vT, idxLoop,name) # grid object
grid # display grid object

idxLoop  idxEm
0        CO2        0.0
1        CO2       25.0
2        CO2       50.0
3        CO2       75.0
4        CO2      100.0
5        CO2      125.0
6        CO2      150.0
7        CO2      175.0
8        CO2      200.0
9        CO2      225.0
10       CO2      250.0
Name: taxEm, dtype: float64

### Manual, efficient implementation

First, we show an efficient implementation (in terms of minimizing compilation time). Because our simulations only involve changing exogenous parameters (and not e.g. expanding indices/domains or introducing new variables/domains/constraints), we do not have to compile (set up model structure) in every iteration of the loop. We can simply adjust the `self.sys.out` dictionary that is ultimately the one that is passed to the solver. While this is efficient in terms of minimizing compilation time, it involves writing more code. 

In our case, the shock/simulation changes the tax on emissions, which changes the marginal costs of generators. Thus, we start by writing a new grid that is expressed in terms of marginal costs instead:

In [4]:
gridmc = mc(m.db('uFuel'), m.db('VOM'), m.db('pFuel'), m.db('uEm'), grid) # this calls the local function "mc"

Next, our shock involves updating the cost parameter in `self.sys.out['c']`, but we only need to update a part of the vector (only the part related to the variable `generation`). We can get the relevant indices by looking up in the `self.sys.maps['v']` dictionary:

In [5]:
vIdx = m.sys.maps['v']['generation'].values # this is the indices in the vector self.sys.out['c'] that relates to the `generation` variable

Now, we can loop through our grid, update the relevant parts of the cost vector, solve, and store the solution:

In [6]:
solsLoop = dict.fromkeys(idxLoop)
for l in idxLoop:
    m.sys.out['c'][vIdx] = gridmc.xs(l, level = idxLoop.name) # update relevant coefficients in the 'c' vector
    solsLoop[l] = m.postSolve(m.solve()) # store solution in dictionary

We can turn this into a dictionary of frames instead using:

In [7]:
solsDF = loopUnpackToDFs(solsLoop, idxLoop)

### Simple, but automated implementation

An alternative to the manual approach is to use the `self.lazyLoop` method (also specified for the `ModelShell` class that we always build on). The loop goes through four steps: (1) Update database values from specified grids, (2) compile model again from scratch, (3) solve, (4) save post-solve routine to dict. 

We refer to this as a "lazy" loop, because it allows us to do a lot with small bits of code, but it spends unnecessary time recompiling the entire model from scratch; recall that once the model is compiled, the `self.sys.out` dictionary stores the relevant arguments that we need to pass to a solver. Often, the shock/simulation we have in mind can be carried out by simply adjusting selected elements in `self.sys.out` and then resolving. 

While this is not necessarily efficient, it is convenient. For our shock above, for instance, we simply call:

In [8]:
solsDF = m.lazyLoopAsDFs(grid, idxLoop)
solsDF['generation'] # inspect some of the solution

idxLoop,0,1,2,3,4,5,6,7,8,9,10
idxGen,,,,,,,,,,,
wind,720.0,720.0,720.0,720.0,720.0,720.0,720.0,720.0,720.0,720.0,720.0
solar,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0
nuclear,3600.0,3600.0,3600.0,3600.0,3600.0,3600.0,3600.0,3600.0,3600.0,3600.0,3600.0
biomass,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0
natgas,1260.0,1260.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
coal,2880.0,2880.0,2880.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
